In [53]:
import numpy as np
import pandas as pd
import pickle as pkl
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split,LeaveOneOut,KFold,RepeatedStratifiedKFold
%matplotlib inline
import matplotlib
#matplotlib.use('agg')
matplotlib.style.use('ggplot')

from matplotlib import pyplot as plt
from collections import Counter
from functools import reduce
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
from nltk.tag.stanford import StanfordNERTagger
st = StanfordNERTagger('unigrams/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz','unigrams/stanford-ner-2018-02-27/stanford-ner-3.9.1.jar',encoding='utf-8')


In [54]:
br_to_us=pd.read_excel("Book.xlsx")
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))
decades=[2000, 1990, 1980, 1970, 1960, 1950, 1940, 1930, 1920, 1900, 1910,
            1890, 1880, 1870, 1850, 1860, 1840, 1830, 1820, 1810, 1800]
old_decades=decades[1:]
l1=dict(zip([1,-1],['Positive','Negative']))
replacements={'modifier':br_to_us_dict,'head':br_to_us_dict,'Class':l1}
new_heads = pkl.load( open( "new_heads.p", "rb" ) )
new_modifiers = pkl.load( open( "new_modifiers.p", "rb" ) )

In [55]:
def tupleconverter(tup):
    dicter={}
    for word,decade in tup:
        #print(word,decade)
        if word in dicter:
            dicter[word].append(decade)
        else:
            dicter[word]=[]
            dicter[word].append(decade)
            
    #for word,decades in dicter.items():
        #print(word)
        #print(decades)
        #dicter[word]=Counter(decades)
        #print(dicter[word])
    return(dicter)


def decremover(dc):

    new_constituents=[]
    for constituent,decades in dc.items():
        if len(decades)==1:
            if 2000 in decades:
                new_constituents.append(constituent)
    return(new_constituents)




def magnituder(x):
    return(np.sqrt(np.sum(np.square(x))))

def new_compound_finder(df,decade=2000):
    new_compounds=[]
    for key,val in compound_decades.items():
        if len(val)==1 and decade in val:
            new_compounds.append(key)
    return new_compounds

def lemma_maker(x):
    #print(x,y)
    return lemmatizer.lemmatize(x)

def ner_checker(stat):
    temp_list=st.tag(stat)
    if temp_list[0][1]=="PERSON" and temp_list[1][1]=="PERSON":
        return stat,True
    else:
        return stat,False


## Loading the individual datasets of Modifiers, Heads and Compounds

In [56]:
heads=pd.read_csv("heads.csv",sep="\t",header=None,usecols=[1,2,4])
heads.columns=["head","context","count"]
#heads.info()
total_head_count=heads['count'].sum()
heads=heads.groupby(['head','context'])['count'].sum().to_frame()
heads.reset_index(inplace=True)
heads=heads.loc[~heads['head'].isin(new_heads)]
heads.set_index(['head'],inplace=True)
heads

,context,count
head,,
a_n,abridge_v,59
a_n,abstract_n,569
a_n,accredited_a,74
a_n,acquire_v,116
a_n,activity_n,105
a_n,acute_a,44
a_n,alarm_n,46
a_n,almanac_n,899
a_n,always_r,509


In [57]:
head_counts=heads.groupby(['head'])['count'].sum().to_frame()
head_counts.columns=['head_count']
head_counts

,head_count
head,
a_n,73116
aa_n,356
aaron_n,984
ab_n,8318
ababa_n,587
abandonment_n,432
abatement_n,350
abba_n,150
abbas_n,732


In [58]:
modifiers=pd.read_csv("modifiers.csv",sep="\t",header=None,usecols=[1,2,4])
modifiers.columns=["modifier","context","count"]
total_modifier_count=modifiers['count'].sum()
modifiers=modifiers.groupby(['modifier','context'])['count'].sum().to_frame()
modifiers.reset_index(inplace=True)
modifiers=modifiers.loc[~modifiers['modifier'].isin(new_modifiers)]
modifiers.set_index(['modifier'],inplace=True)
modifiers

,context,count
modifier,,
a_n,almanac_n,100
a_n,b_n,55
a_n,be_v,185
a_n,bear_v,88
a_n,bell_n,173
a_n,best_a,159
a_n,book_n,44
a_n,broadway_a,88
a_n,c_n,55


In [59]:
modifier_counts=modifiers.groupby(['modifier'])['count'].sum().to_frame()
modifier_counts.columns=['modifier_count']
modifier_counts

,modifier_count
modifier,
a_n,6326
aa_n,2571
aaa_n,147
aaron_n,6724
ab_n,1514
aba_n,2560
ababa_n,224
abatement_n,2076
abbe_n,252


In [60]:
compounds=pd.read_csv("compounds.csv",sep="\t",header=None,usecols=[1,2,3,5])
compounds.columns=["modifier","head","context","count"]
compounds=compounds=compounds.groupby(['modifier','head','context'])['count'].sum().to_frame()
compounds.reset_index(inplace=True)
compounds=compounds.loc[~(compounds.modifier.isin(new_modifiers)&compounds['head'].isin(new_modifiers))]
compounds=compounds.loc[compounds.modifier!=compounds['head']]

compounds['compound']=compounds['modifier'].str[:-2]+" "+compounds['head'].str[:-2]
#compounds['NER']=np.vectorize(ner_checker)(compounds['compound'])
compounds.set_index(['modifier','head'],inplace=True)
compounds

context  count            compound
modifier     head                                                    
a_n          bell_n        international_a    173              a bell
             bell_n            microfilm_n    173              a bell
             bell_n           university_n    173              a bell
             c_n                       b_n     55                 a c
             c_n                      be_v     55                 a c
             c_n                     let_v     55                 a c
             cassell_n             house_n    106           a cassell
             cassell_n           imprint_v    194           a cassell
             cassell_n             press_n     88           a cassell
             cassell_n        university_n     44           a cassell
             cassell_n        wellington_n    150           a cassell
             chorus_n           broadway_a     88            a chorus
             chorus_n               line_n     88            a chorus
             chorus_n            musical_a     88            a chorus
             colonial_n               be_v     42          a colonial
             colonial_n          edition_n     42          a colonial
             colonial_n              net_a     42          a colonial
             man_n               chapter_n     43               a man
             man_n                   out_v     43               a man
             man_n                     v_n     43               a man
             publisher_n            best_a    159         a publisher
             publisher_n            book_n    159         a publisher
             publisher_n          weekly_r    159         a publisher
             sand_n              almanac_n    100              a sand
             sand_n              classic_a    100              a sand
             sand_n               county_n    100              a sand
             series_n            present_v    408            a series
             series_n            science_n    408            a series
             series_n             series_n    408            a series
             star_n                   be_v     88              a star
...                                    ...    ...                 ...
zoning_n     practice_n        challenge_v     73     zoning practice
             practice_n     exclusionary_a    146     zoning practice
             practice_n             must_v    217     zoning practice
             practice_n         specific_a     71     zoning practice
             profile_n               can_v     57      zoning profile
             profile_n          chemical_a     58      zoning profile
             profile_n          detailed_a    115      zoning profile
             profile_n            refine_v     57      zoning profile
             profile_n               use_v     58      zoning profile
             regulation_n          local_a     49   zoning regulation
             regulation_n        require_v     49   zoning regulation
             regulation_n          space_n     49   zoning regulation
zoological_n park_n                   be_v    297     zoological park
             park_n                 have_v     48     zoological park
             park_n                  new_n    345     zoological park
             park_n                 york_n    345     zoological park
             society_n                be_v    174  zoological society
             society_n              have_v    209  zoological society
             society_n               new_n    383  zoological society
             society_n              york_n    383  zoological society
zoom_n       lens_n                 also_r     57           zoom lens
             lens_n                   be_v    126           zoom lens
             lens_n                  can_v    126           zoom lens
             lens_n                  use_v     69           zoom lens
zurich_n     agreement_n              be_v   

In [61]:
compound_counts=compounds.groupby(['modifier','head'])['count'].sum().to_frame()
compound_counts.columns=['compound_count']
compound_counts

compound_count
modifier     head                         
a_n          bell_n                    519
             c_n                       165
             cassell_n                 582
             chorus_n                  264
             colonial_n                126
             man_n                     129
             publisher_n               477
             sand_n                    300
             series_n                 1224
             star_n                    264
             streetcar_n               870
             y_n                        27
aa_n         group_n                  1260
             name_n                   1311
aaa_n        study_n                   147
aaron_n      burr_n                   5034
             douglas_n                 357
             presley_n                 171
             smith_n                   219
             spark_n                   171
             stark_n                   135
ab_n         blood_n                   501
             cd_n                      141
             people_n                  273
             performance_n             195
             tract_n                   126
aba_n        advisory_n                378
             antitrust_n               165
             canon_n                   186
             model_n                   411
...                                    ...
zipper_n     bag_n                     336
zone_n       academy_n                 369
             act_n                     291
             b_n                      1032
             color_n                   465
             commander_n               498
             concept_n                 162
             criterion_n               336
             d_n                       189
             government_n              285
             ii_n                      168
             lymphoma_n                147
             management_n             5169
             product_n                 138
             size_n                   1398
             temperature_n             180
             treaty_n                  126
zoning_n     authority_n               369
             commission_n              120
             interest_n                327
             law_n                     198
             ordinance_n              2043
             practice_n                651
             profile_n                 345
             regulation_n              147
zoological_n park_n                   1035
             society_n                1149
zoom_n       lens_n                    378
zurich_n     agreement_n               132
             government_n              126

[78143 rows x 1 columns]

In [62]:
information_feat=compound_counts
information_feat.columns=['a']
X_star=compounds.groupby(['modifier'])['count'].sum().to_frame()
X_star.columns=['x_star']
Y_star=compounds.groupby(['head'])['count'].sum().to_frame()
Y_star.columns=['star_y']
information_feat=pd.merge(information_feat.reset_index(),X_star.reset_index(),on=['modifier'])
information_feat=pd.merge(information_feat,Y_star.reset_index(),on=['head'])
information_feat['b']=information_feat['x_star']-information_feat['a']
information_feat['c']=information_feat['star_y']-information_feat['a']
information_feat['N']=compound_counts.sum()[0]
information_feat['d']=information_feat['N']-(information_feat['a']+information_feat['b']+information_feat['c'])
information_feat['x_bar_star']=information_feat['N']-information_feat['x_star']
information_feat['star_y_bar']=information_feat['N']-information_feat['star_y']
information_feat.set_index(['modifier','head'],inplace=True)
information_feat.replace(0,0.001,inplace=True)
information_feat['log_ratio']=2*(information_feat['a']*np.log((information_feat['a']*information_feat['N'])/(information_feat['x_star']*information_feat['star_y']))+\
information_feat['b']*np.log((information_feat['b']*information_feat['N'])/(information_feat['x_star']*information_feat['star_y_bar']))+\
information_feat['c']*np.log((information_feat['c']*information_feat['N'])/(information_feat['x_bar_star']*information_feat['star_y']))+\
information_feat['d']*np.log((information_feat['d']*information_feat['N'])/(information_feat['x_bar_star']*information_feat['star_y_bar'])))
information_feat['ppmi']=np.log2((information_feat['a']*information_feat['N'])/(information_feat['x_star']*information_feat['star_y']))
information_feat['local_mi']=information_feat['a']*information_feat['ppmi']
information_feat.ppmi.loc[information_feat.ppmi<=0]=0
information_feat.drop(['a','x_star','star_y','b','c','d','N','d','x_bar_star','star_y_bar'],axis=1,inplace=True)
information_feat

,,log_ratio,ppmi,local_mi
modifier,head,,,
a_n,bell_n,5378.361,8.823,4579.386
alarm_n,bell_n,42169.085,9.950,34475.197
alice_n,bell_n,2123.495,9.189,1791.797
anne_n,bell_n,970.274,6.480,894.268
baby_n,bell_n,850.718,4.977,850.987
bedroom_n,bell_n,906.043,6.722,826.820
brass_n,bell_n,2799.953,7.924,2448.612
canterbury_n,bell_n,1323.286,8.761,1130.172
castle_n,bell_n,1444.815,7.009,1303.627


In [63]:
information_feat.describe()

,log_ratio,ppmi,local_mi
count,78143.000,78143.000,78143.000
mean,13873.818,6.909,10926.867
std,198083.945,4.014,126064.993
min,0.000,0.000,-22997.543
25%,956.022,3.922,944.180
50%,2311.664,6.720,2001.628
75%,5751.565,9.638,4867.354
max,42809567.691,19.968,25489354.556


In [64]:
new_modifiers=modifiers.groupby(['modifier','context'])['count'].sum().to_frame()
new_modifiers.columns=['a']
mod_star=modifiers.groupby(['modifier'])['count'].sum().to_frame()
mod_star.columns=['mod_star']
context_star=modifiers.groupby(['context'])['count'].sum().to_frame()
context_star.columns=['context_star']
merge1=pd.merge(new_modifiers.reset_index(),mod_star.reset_index(),on=['modifier'])
new_modifiers=pd.merge(merge1,context_star.reset_index(),on=['context'])
new_modifiers['N']=new_modifiers['a'].sum()
new_modifiers['modifier_ppmi']=np.log2((new_modifiers['a']*new_modifiers['N'])/(new_modifiers['mod_star']*new_modifiers['context_star']**0.75))
new_modifiers.set_index(['modifier'],inplace=True)
new_modifiers.drop(['a','mod_star','context_star','N'],axis=1,inplace=True)
new_modifiers.modifier_ppmi.loc[new_modifiers.modifier_ppmi<=0]=0
new_modifiers

,context,modifier_ppmi
modifier,,
a_n,almanac_n,12.782
adam_n,almanac_n,10.748
almanac_n,almanac_n,12.156
county_n,almanac_n,9.637
data_n,almanac_n,7.221
employment_n,almanac_n,7.459
job_n,almanac_n,7.538
letter_n,almanac_n,9.402
pennsylvania_n,almanac_n,11.757


In [65]:
new_heads=heads.groupby(['head','context'])['count'].sum().to_frame()
new_heads.columns=['a']
head_star=heads.groupby(['head'])['count'].sum().to_frame()
head_star.columns=['head_star']
context_star=heads.groupby(['context'])['count'].sum().to_frame()
context_star.columns=['context_star']
merge1=pd.merge(new_heads.reset_index(),head_star.reset_index(),on=['head'])
new_heads=pd.merge(merge1,context_star.reset_index(),on=['context'])
new_heads['N']=new_heads['a'].sum()
new_heads['head_ppmi']=np.log2((new_heads['a']*new_heads['N'])/(new_heads['head_star']*new_heads['context_star']**0.75))
new_heads.set_index(['head'],inplace=True)
new_heads.drop(['a','head_star','context_star','N'],axis=1,inplace=True)
new_heads.head_ppmi.loc[new_heads.head_ppmi<=0]=0
new_heads

,context,head_ppmi
head,,
a_n,abridge_v,9.208
certificate_n,abridge_v,10.567
congress_n,abridge_v,8.340
freedom_n,abridge_v,13.575
interest_n,abridge_v,7.570
matter_n,abridge_v,9.490
right_n,abridge_v,8.134
state_n,abridge_v,3.836
a_n,abstract_n,11.966


In [66]:
new_compounds=compounds.groupby(['modifier','head','context'])['count'].sum().to_frame()
new_compounds.columns=['a']
compound_star=compounds.groupby(['modifier','head'])['count'].sum().to_frame()
compound_star.columns=['compound_star']
context_star=compounds.groupby(['context'])['count'].sum().to_frame()
context_star.columns=['context_star']
new_compounds=pd.merge(new_compounds.reset_index(),compound_star.reset_index(),on=['modifier','head'])
new_compounds=pd.merge(new_compounds,context_star.reset_index(),on=['context'])
new_compounds['N']=new_compounds['a'].sum()
new_compounds['compound_ppmi']=np.log2((new_compounds['a']*new_compounds['N'])/(new_compounds['compound_star']*new_compounds['context_star']**0.75))
new_compounds.set_index(['modifier','head'],inplace=True)
new_compounds.drop(['a','compound_star','context_star','N'],axis=1,inplace=True)
new_compounds.compound_ppmi.loc[new_compounds.compound_ppmi<=0]=0
new_compounds

context  compound_ppmi
modifier      head                                         
a_n           bell_n         international_a         13.227
academic_n    advisory_n     international_a         13.227
accounting_n  standard_n     international_a          8.200
acquisition_n workshop_n     international_a         13.227
aid_n         agency_n       international_a          8.578
air_n         navigation_n   international_a         12.764
              service_n      international_a         11.441
              transport_n    international_a         10.742
arbitral_n    institution_n  international_a         13.227
arbitration_n agreement_n    international_a          8.043
arm_n         trade_n        international_a         13.227
armistice_n   commission_n   international_a          8.542
arrest_n      warrant_n      international_a          8.346
art_n         education_n    international_a          9.107
atmosphere_n  research_n     international_a         13.227
ballantine_n  book_n         international_a         12.545
bank_n        lending_n      international_a         11.642
banking_n     system_n       international_a          6.184
black_n       hill_n         international_a         11.141
bone_n        marrow_n       international_a          4.611
book_n        search_n       international_a         13.227
              series_n       international_a          7.871
boundary_n    land_n         international_a         13.227
              road_n         international_a         13.227
brand_n       recognition_n  international_a         13.227
bretton_n     wood_n         international_a         12.018
business_n    activity_n     international_a          8.131
              perspective_n  international_a         13.227
              practice_n     international_a         10.769
              series_n       international_a         13.227
...                                      ...            ...
wyoming_n     stock_n               grower_n         20.824
x_n           file_n                 eater_n         20.397
              height_n       parallelogram_n         18.532
              ray_n                    tem_n         15.082
              y_n                        z_v         19.415
yankee_n      jim_n              wherefore_r         21.101
              league_n            audacity_n         18.952
year_n        book_n                   law_a         15.719
              chartres_n           pilgrim_n         20.859
              mark_n                tucker_n         20.118
              poet_n                 weary_v         21.195
yeast_n       infection_n             jock_n         20.740
yesterday_n   afternoon_n           stroll_n         18.243
york_n        banker_n                 sip_v         19.386
              chichester_n        brisbane_n         19.912
              city_n            experiment_v          9.268
              city_n                harlem_n          9.458
              city_n                  slum_a          9.292
              hamilton_n             stone_v         21.245
              state_n         correctional_a         12.490
              state_n            shorthand_n         11.897
              time_n                   dub_v         10.851
              time_n              pentagon_n         11.101
young_n       award_n                   cy_n         21.147
              commission_n          hilton_n         21.147
              day_n                  shade_v         20.693
              green_n                blest_a         21.271
              jerry_n        superstitious_a         19.051
youth_n       sexuality_n      frustrating_a         19.208
zipper_n      bag_n                    tan_a         19.934

[507425 rows x 2 columns]

In [68]:
modifier_denom=new_modifiers.groupby(['modifier'])['modifier_ppmi'].agg(lambda x: np.sqrt(np.sum(np.square(x)))).to_frame()
modifier_denom.columns=['modifier_denom']
modifier_denom

,modifier_denom
modifier,
a_n,77.354
aa_n,27.615
aaa_n,25.453
aaron_n,62.945
ab_n,50.903
aba_n,49.411
ababa_n,28.419
abatement_n,33.624
abbe_n,24.109


In [69]:
head_denom=new_heads.groupby(['head'])['head_ppmi'].agg(lambda x: np.sqrt(np.sum(np.square(x)))).to_frame()
head_denom.columns=['head_denom']
head_denom

,head_denom
head,
a_n,119.832
aa_n,32.724
aaron_n,39.766
ab_n,67.987
ababa_n,38.792
abandonment_n,18.500
abatement_n,27.165
abba_n,25.134
abbas_n,33.879


In [70]:
compound_denom=new_compounds.groupby(['modifier','head'])['compound_ppmi'].agg(lambda x: np.sqrt(np.sum(np.square(x)))).to_frame()
compound_denom.columns=['compound_denom']
compound_denom

compound_denom
modifier     head                         
a_n          bell_n                 27.114
             c_n                    21.745
             cassell_n              33.788
             chorus_n               29.180
             colonial_n             21.125
             man_n                  28.819
             publisher_n            26.288
             sand_n                 28.849
             series_n               24.723
             star_n                 20.248
             streetcar_n            35.225
             y_n                    21.809
aa_n         group_n                27.225
             name_n                 22.332
aaa_n        study_n                29.614
aaron_n      burr_n                 50.396
             douglas_n              25.274
             presley_n              25.011
             smith_n                26.890
             spark_n                18.082
             stark_n                25.747
ab_n         blood_n                28.700
             cd_n                   25.962
             people_n               27.116
             performance_n          26.618
             tract_n                28.925
aba_n        advisory_n             22.257
             antitrust_n            22.974
             canon_n                30.422
             model_n                21.086
...                                    ...
zipper_n     bag_n                  27.093
zone_n       academy_n              25.760
             act_n                  20.667
             b_n                    30.575
             color_n                34.120
             commander_n            30.005
             concept_n              21.830
             criterion_n            24.156
             d_n                    17.875
             government_n           31.398
             ii_n                   29.416
             lymphoma_n             29.335
             management_n           40.834
             product_n              19.010
             size_n                 41.224
             temperature_n          20.597
             treaty_n               23.410
zoning_n     authority_n            24.985
             commission_n           22.466
             interest_n             22.023
             law_n                  24.961
             ordinance_n            47.892
             practice_n             31.770
             profile_n              30.335
             regulation_n           23.504
zoological_n park_n                 18.185
             society_n              18.550
zoom_n       lens_n                 17.667
zurich_n     agreement_n            21.826
             government_n           28.243

[78143 rows x 1 columns]

In [71]:
compound_modifier_sim=pd.merge(new_compounds.reset_index(),new_modifiers.reset_index(),on=["modifier","context"])
compound_modifier_sim['numerator']=compound_modifier_sim['compound_ppmi']*compound_modifier_sim['modifier_ppmi']
compound_modifier_sim=compound_modifier_sim.groupby(['modifier','head'])['numerator'].sum().to_frame()
compound_modifier_sim=pd.merge(compound_modifier_sim.reset_index(),compound_denom.reset_index(),on=["modifier","head"])
compound_modifier_sim=pd.merge(compound_modifier_sim,modifier_denom.reset_index(),on=['modifier'])
compound_modifier_sim['sim_with_modifier']=compound_modifier_sim['numerator']/(compound_modifier_sim['compound_denom']*compound_modifier_sim['modifier_denom'])
compound_modifier_sim.set_index(['modifier','head'],inplace=True)
compound_modifier_sim.drop(['numerator','compound_denom'],axis=1,inplace=True)
compound_modifier_sim

modifier_denom  sim_with_modifier
modifier     head                                            
a_n          bell_n                 77.354              0.265
             c_n                    77.354              0.173
             cassell_n              77.354              0.291
             chorus_n               77.354              0.274
             colonial_n             77.354              0.152
             man_n                  77.354              0.245
             publisher_n            77.354              0.249
             sand_n                 77.354              0.274
             series_n               77.354              0.271
             star_n                 77.354              0.180
             streetcar_n            77.354              0.362
             y_n                    77.354              0.139
aa_n         group_n                27.615              0.610
             name_n                 27.615              0.744
aaa_n        study_n                25.453              0.608
aaron_n      burr_n                 62.945              0.535
             douglas_n              62.945              0.223
             presley_n              62.945              0.287
             smith_n                62.945              0.318
             spark_n                62.945              0.196
             stark_n                62.945              0.287
ab_n         blood_n                50.903              0.528
             cd_n                   50.903              0.399
             people_n               50.903              0.287
             performance_n          50.903              0.415
             tract_n                50.903              0.426
aba_n        advisory_n             49.411              0.372
             antitrust_n            49.411              0.365
             canon_n                49.411              0.477
             model_n                49.411              0.353
...                                    ...                ...
zipper_n     bag_n                  28.136              0.684
zone_n       academy_n              78.668              0.200
             act_n                  78.668              0.166
             b_n                    78.668              0.292
             color_n                78.668              0.322
             commander_n            78.668              0.152
             concept_n              78.668              0.168
             criterion_n            78.668              0.191
             d_n                    78.668              0.062
             government_n           78.668              0.241
             ii_n                   78.668              0.224
             lymphoma_n             78.668              0.271
             management_n           78.668              0.457
             product_n              78.668              0.147
             size_n                 78.668              0.358
             temperature_n          78.668              0.156
             treaty_n               78.668              0.184
zoning_n     authority_n            65.616              0.201
             commission_n           65.616              0.124
             interest_n             65.616              0.124
             law_n                  65.616              0.276
             ordinance_n            65.616              0.547
             practice_n             65.616              0.341
             profile_n              65.616              0.348
             regulation_n           65.616              0.236
zoological_n park_n                 25.009              0.708
             society_n              25.009              0.709
zoom_n       lens_n                 21.468              0.594
zurich_n     agreement_n            31.970              0.381
             government_n           31.970              0.602

[77993 rows x 2 columns]

In [73]:
compound_head_sim=pd.merge(new_compounds.reset_index(),new_heads.reset_index(),on=["head","context"])

compound_head_sim['numerator']=compound_head_sim['compound_ppmi']*compound_head_sim['head_ppmi']
compound_head_sim=compound_head_sim.groupby(['modifier','head'])['numerator'].sum().to_frame()
compound_head_sim=pd.merge(compound_head_sim.reset_index(),compound_denom.reset_index(),on=["modifier","head"])
compound_head_sim=pd.merge(compound_head_sim,head_denom.reset_index(),on=['head'])
compound_head_sim['sim_with_head']=compound_head_sim['numerator']/(compound_head_sim['compound_denom']*compound_head_sim['head_denom'])
compound_head_sim.set_index(['modifier','head'],inplace=True)
compound_head_sim.drop(['numerator','compound_denom'],axis=1,inplace=True)
compound_head_sim

,,head_denom,sim_with_head
modifier,head,,
a_n,bell_n,116.903,0.106
alarm_n,bell_n,116.903,0.237
alice_n,bell_n,116.903,0.112
anne_n,bell_n,116.903,0.141
baby_n,bell_n,116.903,0.052
bedroom_n,bell_n,116.903,0.140
brass_n,bell_n,116.903,0.166
canterbury_n,bell_n,116.903,0.101
castle_n,bell_n,116.903,0.124


In [76]:
constituent_sim=pd.merge(new_heads.reset_index(),new_compounds.reset_index(),on=["head","context"])
constituent_sim=pd.merge(constituent_sim,new_modifiers.reset_index(),on=["modifier","context"])
constituent_sim['numerator']=constituent_sim['head_ppmi']*constituent_sim['modifier_ppmi']
constituent_sim=constituent_sim.groupby(['modifier','head'])['numerator'].sum().to_frame()
constituent_sim=pd.merge(constituent_sim.reset_index(),head_denom.reset_index(),on=["head"])
constituent_sim=pd.merge(constituent_sim,modifier_denom.reset_index(),on=["modifier"])
constituent_sim['sim_bw_constituents']=constituent_sim['numerator']/(constituent_sim['head_denom']*constituent_sim['modifier_denom'])
constituent_sim.set_index(['modifier','head'],inplace=True)
constituent_sim.drop(['numerator','modifier_denom','head_denom'],axis=1,inplace=True)
constituent_sim

sim_bw_constituents
modifier    head                              
a_n         bell_n                       0.028
            c_n                          0.021
            cassell_n                    0.266
            chorus_n                     0.141
            colonial_n                   0.125
            man_n                        0.017
            publisher_n                  0.034
            sand_n                       0.058
            series_n                     0.024
            star_n                       0.018
            streetcar_n                  0.272
            y_n                          0.026
alarm_n     bell_n                       0.111
            rate_n                       0.040
            service_n                    0.007
            system_n                     0.035
            option_n                     0.044
            a_n                          0.023
            box_n                        0.011
            limit_n                      0.021
            barking_n                    0.087
            clock_n                      0.071
            fire_n                       0.029
            gun_n                        0.044
alice_n     bell_n                       0.043
            lee_n                        0.053
            egerton_n                    0.310
            montague_n                   0.064
            vane_n                       0.044
anne_n      bell_n                       0.014
...                                        ...
sinus_n     bradycardia_n                1.000
weller_n    dialect_n                    0.279
slant_n     sunbeam_n                    0.370
smoke_n     blackbird_n                  0.589
sri_n       aurobindo_n                  0.221
            lanka_n                      0.756
stamford_n  raffle_n                     0.753
stern_n     sloan_n                      0.663
stricture_n dilation_n                   0.630
stupidity_n galton_n                     0.482
surgical_n  papyrus_n                    0.578
swami_n     vivekananda_n                0.456
sweden_n    switzerland_n                0.736
syringe_n   plunger_n                    0.390
tel_n       aviv_n                       0.531
theodor_n   herzl_n                      0.562
thurston_n  peck_n                       0.579
wasp_n      sting_n                      0.232
tit_n       clutch_n                     0.602
ty_n        cobb_n                       0.575
tyranny_n   contempt_n                   0.350
utopia_n    postulate_n                  0.262
vena_n      cava_n                       0.927
verdict_n   loser_n                      0.351
vincent_n   plum_n                       0.651
vitality_n  subculture_n                 0.460
wendy_n     haley_n                      0.513
wentworth_n higginson_n                  0.568
wilcox_n    kitten_n                     0.195
wireless_n  telephony_n                  0.455

[77870 rows x 1 columns]

In [85]:
dfs = [constituent_sim.reset_index(), compound_head_sim.reset_index(), compound_modifier_sim.reset_index(), information_feat.reset_index()]
compounds_final = reduce(lambda left,right: pd.merge(left,right,on=['modifier','head']), dfs)
compounds_final.drop(['head_denom','modifier_denom'],axis=1,inplace=True)
compounds_final.set_index(['modifier','head'],inplace=True)
compounds_final.fillna(0,inplace=True)
compounds_final -= compounds_final.min()
compounds_final /= compounds_final.max()
compounds_final

sim_bw_constituents  sim_with_head  \
modifier    head                                                
a_n         bell_n                       0.028          0.106   
            c_n                          0.021          0.124   
            cassell_n                    0.266          0.911   
            chorus_n                     0.141          0.513   
            colonial_n                   0.125          0.820   
            man_n                        0.017          0.070   
            publisher_n                  0.034          0.135   
            sand_n                       0.058          0.209   
            series_n                     0.024          0.089   
            star_n                       0.018          0.101   
            streetcar_n                  0.272          0.752   
            y_n                          0.026          0.188   
alarm_n     bell_n                       0.111          0.237   
            rate_n                       0.040          0.118   
            service_n                    0.007          0.061   
            system_n                     0.035          0.098   
            option_n                     0.044          0.163   
            a_n                          0.023          0.128   
            box_n                        0.011          0.045   
            limit_n                      0.021          0.123   
            barking_n                    0.087          0.814   
            clock_n                      0.071          0.332   
            fire_n                       0.029          0.118   
            gun_n                        0.044          0.206   
alice_n     bell_n                       0.043          0.112   
            lee_n                        0.053          0.145   
            egerton_n                    0.310          0.802   
            montague_n                   0.064          0.221   
            vane_n                       0.044          0.177   
anne_n      bell_n                       0.014          0.141   
...                                        ...            ...   
sinus_n     bradycardia_n                1.000          1.000   
weller_n    dialect_n                    0.279          0.437   
slant_n     sunbeam_n                    0.370          0.782   
smoke_n     blackbird_n                  0.589          0.845   
sri_n       aurobindo_n                  0.221          0.907   
            lanka_n                      0.756          0.985   
stamford_n  raffle_n                     0.753          0.866   
stern_n     sloan_n                      0.663          0.808   
stricture_n dilation_n                   0.631          0.727   
stupidity_n galton_n                     0.482          0.813   
surgical_n  papyrus_n                    0.578          0.768   
swami_n     vivekananda_n                0.456          0.856   
sweden_n    switzerland_n                0.736          0.902   
syringe_n   plunger_n                    0.390          0.739   
tel_n       aviv_n                       0.531          0.885   
theodor_n   herzl_n                      0.562          0.739   
thurston_n  peck_n                       0.579          0.819   
wasp_n      sting_n                      0.232          0.615   
tit_n       clutch_n                     0.602          0.774   
ty_n        cobb_n                       0.575          0.703   
tyranny_n   contempt_n                   0.350          0.776   
utopia_n    postulate_n                  0.262          0.468   
vena_n      cava_n                       0.927          0.977   
verdict_n   loser_n                      0.351          0.761   
vincent_n   plum_n                       0.651          0.820   
vitality_n  subculture_n                 0.460          0.729   
wendy_n     haley_n                      0.514          0.848   
wentworth_n higginson_n                  0.569          0.754   
wilcox_n    kitten_n                     0.195          0.687 

In [86]:
graves13_study=pd.read_excel("13428_2012_256_MOESM1_ESM.xlsx",usecols=[0,1,2,3])
graves13_study['modifier'],graves13_study['head']=graves13_study['Stim'].str.split('-', 1).str
graves13_study['modifier']=np.vectorize(lemma_maker)(graves13_study['modifier'])
graves13_study['head']=np.vectorize(lemma_maker)(graves13_study['head'])
graves13_study.replace(replacements,inplace=True)
graves13_study['modifier']=graves13_study['modifier']+"_n"
graves13_study['head']=graves13_study['head']+"_n"
graves13_study.drop(["Stim"],axis=1,inplace=True)
graves13_study.set_index(['modifier','head'],inplace=True)
graves13_study

Class  Mean_rate    SD
modifier   head                                  
bus_n      bridge_n     Positive      1.345 1.396
           window_n     Positive      3.828 0.539
           home_n       Positive      1.828 1.605
bike_n     pant_n       Positive      2.897 1.472
           barn_n       Positive      1.429 1.425
           leg_n        Positive      1.138 1.356
           home_n       Positive      1.483 1.595
           headlight_n  Positive      3.448 0.985
           seat_n       Positive      3.793 0.774
police_n   dog_n        Positive      4.000 0.000
           star_n       Positive      2.034 1.742
           car_n        Positive      4.000 0.000
           horse_n      Positive      3.690 0.806
           truck_n      Positive      3.724 0.528
           telephone_n  Positive      3.241 1.215
           radio_n      Positive      3.724 0.922
           bomb_n       Positive      2.586 1.500
           vehicle_n    Positive      4.000 0.000
           tape_n       Positive      3.655 0.974
           van_n        Positive      3.862 0.581
cab_n      car_n        Positive      2.966 1.295
           truck_n      Positive      1.345 1.203
           window_n     Positive      3.793 0.559
           home_n       Positive      1.724 1.533
sled_n     dog_n        Positive      3.552 0.948
chimp_n    mother_n     Positive      3.103 1.175
beach_n    sunset_n     Positive      3.310 1.228
           ball_n       Positive      4.000 0.000
           house_n      Positive      3.862 0.441
           sand_n       Positive      3.250 1.206
...                          ...        ...   ...
cave_n     mountain_n   Negative      1.786 1.287
winter_n   mountain_n   Negative      2.214 1.397
golf_n     mountain_n   Negative      1.107 1.166
lake_n     mountain_n   Negative      1.071 1.184
goat_n     mountain_n   Negative      1.786 1.475
road_n     mountain_n   Negative      0.857 1.113
meadow_n   mountain_n   Negative      1.286 1.243
building_n mountain_n   Negative      0.357 0.621
face_n     mountain_n   Negative      0.429 0.742
ear_n      elephant_n   Negative      0.786 0.876
boot_n     car_n        Negative      0.714 0.976
nut_n      car_n        Negative      0.714 1.084
window_n   car_n        Negative      1.071 0.900
home_n     car_n        Negative      1.536 1.427
radio_n    car_n        Negative      1.107 0.956
body_n     car_n        Negative      1.107 1.227
phone_n    car_n        Negative      0.964 1.401
bomb_n     car_n        Negative      1.500 1.552
magazine_n car_n        Negative      0.607 0.786
building_n car_n        Negative      0.786 1.067
plant_n    car_n        Negative      0.333 0.877
seat_n     car_n        Negative      1.214 1.343
engine_n   car_n        Negative      1.286 1.301
bed_n      stream_n     Negative      0.571 1.034
           ocean_n      Negative      0.786 1.031
water_n    ocean_n      Negative      1.179 1.156
swimming_n ocean_n      Negative      1.643 1.193
surf_n     ocean_n      Negative      1.000 0.943
house_n    milk_n       Negative      1.357 1.224
coffee_n   milk_n       Negative      2.036 1.374

[2160 rows x 3 columns]

In [87]:
compounds_final_graves=pd.merge(graves13_study.reset_index(),compounds_final.reset_index(),on=['modifier','head'])
#compounds_final.set_index(['modifier','head'],inplace=True)
compounds_final_graves.set_index(['modifier','head'],inplace=True)
compounds_final_graves

Class  Mean_rate    SD  sim_bw_constituents  \
modifier     head                                                         
police_n     car_n       Positive      4.000 0.000                0.047   
             tape_n      Positive      3.655 0.974                0.009   
cab_n        home_n      Positive      1.724 1.533                0.015   
beach_n      sand_n      Positive      3.250 1.206                0.041   
television_n picture_n   Positive      3.690 0.712                0.031   
field_n      kitchen_n   Positive      0.793 1.177                0.016   
             hospital_n  Positive      2.138 1.663                0.012   
             mouse_n     Positive      3.621 1.147                0.026   
             army_n      Positive      2.690 1.285                0.005   
dog_n        book_n      Positive      2.517 1.661                0.005   
water_n      tower_n     Positive      4.000 0.000                0.007   
             pipe_n      Positive      3.724 0.882                0.038   
pant_n       leg_n       Positive      2.828 1.513                0.188   
cigarette_n  butt_n      Positive      3.724 0.882                0.030   
             smoke_n     Positive      3.931 0.371                0.216   
             money_n     Positive      3.034 1.426                0.013   
fire_n       ant_n       Positive      3.483 1.184                0.040   
             bell_n      Positive      3.069 1.334                0.015   
             truck_n     Positive      3.862 0.743                0.009   
             engine_n    Positive      3.793 0.620                0.040   
automobile_n engine_n    Positive      4.000 0.000                0.011   
star_n       field_n     Positive      1.310 1.514                0.016   
blood_n      stream_n    Positive      3.517 1.153                0.013   
             sugar_n     Positive      3.429 1.345                0.258   
             alcohol_n   Positive      3.034 1.451                0.051   
             money_n     Positive      2.276 1.601                0.004   
mountain_n   bike_n      Positive      4.000 0.000                0.070   
             stream_n    Positive      3.483 1.090                0.075   
             lion_n      Positive      3.828 0.658                0.091   
             forest_n    Positive      3.103 1.291                0.011   
...                           ...        ...   ...                  ...   
school_n     building_n  Positive      3.963 0.192                0.096   
             book_n      Positive      3.857 0.591                0.006   
grass_n      house_n     Positive      2.321 1.416                0.008   
picture_n    book_n      Positive      3.929 0.262                0.039   
crown_n      jewel_n     Positive      3.321 0.983                0.017   
             prince_n    Positive      2.667 1.544                0.083   
coffin_n     lid_n       Positive      3.750 0.701                0.273   
cabin_n      boy_n       Positive      3.111 1.251                0.071   
             window_n    Positive      3.929 0.262                0.042   
             door_n      Positive      3.964 0.189                0.047   
street_n     car_n       Positive      3.857 0.448                0.032   
rib_n        cage_n      Positive      3.893 0.567                0.338   
pendulum_n   clock_n     Positive      3.714 0.713                0.140   
bible_n      school_n    Positive      3.821 0.390                0.025   
army_n       hospital_n  Positive      3.714 0.713                0.008   
witch_n      doctor_n    Positive      3.679 0.772                0.074   
trash_n      bag_n       Positive      4.000 0.000                0.198   
sleigh_n     bell_n      Positive      3.857 0.756                0.052   
gold_n       leaf_n      Positive      3.630 0.926                0.007   
bubble_n     gum_n       Positive      4.000 0.000                0.304   
plant_n      water_n     Positive      2.429 1.526   

In [89]:
reddy11_study=pd.read_csv("ijcnlp_compositionality_data/MeanAndDeviations.clean.txt",sep="\t")
#print(reddy11_study.columns)
reddy11_study.columns=['compund','to_divide']
reddy11_study['modifier_mean'],reddy11_study['modifier_std'],reddy11_study['head_mean'],reddy11_study['head_std'],reddy11_study['compound_mean'],reddy11_study['compound_std'],_=reddy11_study.to_divide.str.split(" ",7).str
reddy11_study['modifier'],reddy11_study['head']=reddy11_study['compund'].str.split(" ",2).str
reddy11_study.modifier=reddy11_study.modifier.str[:-2]
reddy11_study['head']=reddy11_study['head'].str[:-2]
reddy11_study.drop(['compund','to_divide'],axis=1,inplace=True)
reddy11_study['modifier']=np.vectorize(lemma_maker)(reddy11_study['modifier'])
reddy11_study['head']=np.vectorize(lemma_maker)(reddy11_study['head'])
reddy11_study.replace(replacements,inplace=True)
reddy11_study['modifier']=reddy11_study['modifier']+"_n"
reddy11_study['head']=reddy11_study['head']+"_n"
reddy11_study

,modifier_mean,modifier_std,head_mean,head_std,compound_mean,compound_std,modifier,head
0,3.866667,1.117537,4.866667,0.339935,4.250000,0.871165,end_n,user_n
1,1.607143,1.654848,1.892857,1.496169,1.703704,1.717337,firing_n,line_n
2,2.821429,1.964935,4.862069,0.344828,3.827586,1.233693,game_n,plan_n
3,4.766667,0.422953,4.862069,0.344828,4.800000,0.476095,application_n,form_n
4,0.600000,0.800000,4.586207,1.099129,1.310345,1.020596,snail_n,mail_n
5,2.678571,1.691440,3.933333,1.181336,3.785714,1.205853,web_n,site_n
6,0.379310,0.805746,4.714286,0.839096,1.517241,1.133219,flea_n,market_n
7,0.428571,0.775913,5.000000,0.000000,2.642857,1.315139,grandfather_n,clock_n
8,1.066667,1.289272,3.407407,1.340516,1.703704,1.047566,diamond_n,wedding_n
9,4.714286,0.589015,0.862069,0.936764,2.185185,1.155888,guilt_n,trip_n


In [92]:
compounds_final_reddy=pd.merge(reddy11_study,compounds_final.reset_index(),on=['modifier','head'])
compounds_final_reddy.set_index(['modifier','head'],inplace=True)
compounds_final_reddy

,,modifier_mean,modifier_std,head_mean,head_std,compound_mean,compound_std,sim_bw_constituents,sim_with_head,sim_with_modifier,log_ratio,ppmi,local_mi
modifier,head,,,,,,,,,,,,
end_n,user_n,3.866667,1.117537,4.866667,0.339935,4.250000,0.871165,0.045,0.292,0.146,0.001,0.375,0.003
firing_n,line_n,1.607143,1.654848,1.892857,1.496169,1.703704,1.717337,0.032,0.068,0.429,0.000,0.353,0.001
application_n,form_n,4.766667,0.422953,4.862069,0.344828,4.800000,0.476095,0.055,0.265,0.200,0.005,0.467,0.007
web_n,site_n,2.678571,1.691440,3.933333,1.181336,3.785714,1.205853,0.365,0.576,0.632,0.018,0.484,0.023
silver_n,spoon_n,1.592593,1.472193,1.444444,1.770820,1.518519,1.449658,0.048,0.700,0.067,0.000,0.548,0.001
rush_n,hour_n,3.107143,1.371633,2.862069,1.357588,3.333333,1.273665,0.138,0.169,0.760,0.000,0.583,0.001
mailing_n,list_n,4.666667,0.537484,4.933333,0.249444,4.666667,0.471405,0.099,0.213,0.465,0.000,0.497,0.001
spelling_n,bee_n,4.814815,0.771580,0.517241,1.037925,2.448276,1.248066,0.071,0.286,0.241,0.000,0.621,0.001
public_n,service_n,4.666667,0.649786,4.766667,0.615540,4.400000,0.757188,0.025,0.101,0.228,0.000,0.099,0.001


In [93]:
with open('compounds_final_graves.csv', 'w') as f1:
    compounds_final_graves.to_csv(f1, header=True,sep="\t")

In [94]:
with open('compounds_final_reddy.csv', 'w') as f1:
    compounds_final_reddy.to_csv(f1, header=True,sep="\t")